In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:12pt; line-height:40px;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# <span style="color:rgb(0,200,100);">1. 패키지 설치 </span>

In [19]:
# python-dotenv, langchain(1.2.0),  langchain-openai,  langchain-pinecone,
# pandas,  langchain-community, docx2txt,  langchain-text_splitters,  langchain_ollama
%pip install -q cohere

Note: you may need to restart the kernel to use updated packages.


# <span style="color:rgb(0,200,100);">2. 환경설정(환경변수, 시스템파라미터변수) </span>

In [74]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_LLM_MODEL = "gpt-4o-mini"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-large" # 차원수 3072

PINECONE_INDEX_NAME = "better-rag-index"
PINECONE_INDEX_DEMENSION = 3072
PINECONE_INDEX_METRIC = "cosine"
PINECONE_INDEX_REGION = "us-east-1"
PINECONE_INDEX_CLOUD = "aws"

COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# <span style="color:rgb(0,200,100);">3. category 추출하기</span>
- 청크 내용의 카테고리, 청크내용의 title, 조항

In [33]:
def get_category():
    return {
        '납세의무': {
            '납세의무': 3, '거주자': 3, '비거주자': 3, '납세의무자': 3, 
            '원천징수': 2, '원천징수의무자': 2, '공동사업자': 2, 
            '상속인': 2, '증여자': 2, '신탁재산': 2
        },

        '세율계산': {
            '세율': 3, '소득세': 3, '과세표준': 3, '산출세액': 3, '세액': 3,
            '결정세액': 2, '세액계산': 2, '기본세율': 2, '세율적용': 2,
            '누진세율': 2, '종합소득세': 2
        },

        '근로소득': {
            '근로소득': 3, '총급여': 3, '급여': 3, '연봉': 3, '임금': 3,
            '근로소득금액': 2, '총급여액': 2, '상여': 2, '수당': 2,
            '봉급': 2, '직장인': 2
        },

        '사업소득': {
            '사업소득': 3, '총수입금액': 3, '필요경비': 3,
            '사업자': 2, '사업소득금액': 2, '결손금': 2, '이월결손금': 2,
            '주택임대소득': 2, '공동사업': 2
        },

        '이자배당': {
            '이자소득': 3, '배당소득': 3, '예금이자': 2, '채권': 2,
            '의제배당': 2, '배당세액공제': 2, '분리과세이자소득': 2,
            '분리과세배당소득': 2
        },

        '양도소득': {
            '양도소득': 3, '자산양도': 2, '부동산양도': 2, '주식양도': 2,
            '양도차익': 2, '취득가액': 2, '양도가액': 2, '양도소득금액': 2
        },

        '연금소득': {
            '연금소득': 3, '연금계좌': 3, '연금저축': 3,
            '퇴직연금': 2, '공적연금': 2, '사적연금': 2, '연금보험': 2,
            '연금수령': 2
        },

        '기타소득': {
            '기타소득': 3, '가상자산': 3, '가상자산소득': 2,
            '상금': 2, '보상금': 2, '종교인소득': 2, '원고료': 2,
            '복권': 1, '당첨금': 1, '발명보상금': 1
        },

        '공제감면': {
            '공제': 3, '소득공제': 3, '세액공제': 3,
            '기본공제': 2, '인적공제': 2, '특별공제': 2, '추가공제': 2,
            '근로소득공제': 2, '연금소득공제': 2, '퇴직소득공제': 2,
            '연금계좌세액공제': 2, '감면': 2
        },

        '비과세': {
            '비과세': 3, '비과세소득': 3, '면제': 2, '세액감면': 2,
            '소득세면제': 2, '복무급여': 1, '실업급여': 1, 
            '출산휴가급여': 1, '장학금': 1
        },

        '신고납부': {
            '신고': 3, '확정신고': 3, '과세표준확정신고': 3, 
            '납부': 3, '중간예납': 2, '가산세': 2,
            '신고기한': 2, '납부기한': 2, '납세지': 2
        },

        '과세기간': {
            '과세기간': 3, '과세연도': 3, '사업연도': 2, 
            '과세기간종료일': 2
        },

        '과세방식': {
            '종합과세': 3, '분리과세': 3, '합산과세': 2, 
            '종합소득과세표준': 2, '분리과세소득': 2, 
            '금융투자소득': 2
        },

        '장부기장': {
            '장부': 3, '복식부기': 3, '간편장부': 2, '기장': 2,
            '장부기록': 2, '증명서류': 2, '기장세액공제': 2
        }
    }

In [34]:
content = """1. 원천징수하는 자가 거주자인 경우: 그 거주자의 주된 사업장 소재지. 다만, 주된 사업장 외의 사업장에서 원천징수를 하는 경우에는 그 사업장의 소재지, 사업장이 없는 경우에는 그 거주자의 주소지 또는 거소지로 한다.
2. 원천징수하는 자가 비거주자인 경우: 그 비거주자의 주된 국내사업장 소재지. 다만, 주된 국내사업장 외의 국내사업장에서 원천징수를 하는 경우에는 그 국내사업장의 소재지, 국내사업장이 없는 경우에는 그 비거주자의 거류지(居留地) 또는 체류지로 한다.
3. 종합과세 분리과세 합산과세 종합소득과세표준 분리과세소득 금융투자소득
"""

In [98]:
def categorize_content(content, top_k=None):
    """내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters :
    - content : 분류할 텍스트 내용
    - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
    Returns:
    - 카테고리 리스트(점수 높은 순)"""
    category_keywords = get_category()
    category_scores = {}
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        # print(category, weighted_keywords)
        score = 0
        for keyword, weight in weighted_keywords.items():
            if keyword in content:
                score += weight
            #count = content.count(keyword) 나온 횟수로 
            #score += count * weight
        if score > 0:
            category_scores[category] = score
    # print(category_scores)
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x:x[1], reverse=True) # 내림차순
    # print(sorted_categories)
    all_categories = [category[0] for category in sorted_categories]
    # print(all_categories)
    # 매칭되는 카테고리가 없으면 "기타" 반환
    if not all_categories:
        all_categories = ["기타"]
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    else:
        return all_categories
categorize_content(content)

['과세방식', '납세의무', '세율계산']

In [36]:
if __name__ == "__main__":
    print(categorize_content("연봉 5000만원인 회사원의 소득세는 얼마예요?"))

['세율계산', '근로소득']


# <span style="color:rgb(0,200,100);">4. langchain 답변 생성 </span>

In [100]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

In [101]:
# 1. LLM과 임베딩 초기화
load_dotenv()
llm = ChatOpenAI(model = OPENAI_LLM_MODEL)
embedding = OpenAIEmbeddings(
    model=OPENAI_EMBEDDING_MODEL,
    #openai_api_key=OPENAI_API_KEY
)
# 2. # 업로드한 벡터db를 가져올 때
pc = Pinecone(api_key=PINECONE_API_KEY)
vector_database = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=PINECONE_INDEX_NAME
)

In [102]:
# 3. 키워드 사전 활용
keyword_dict  = [
    "사람을 나타내는 표현 -> 거주자",
    "직장인 -> 근로소득이 있는 거주자", 
    "월급쟁이 -> 근로소득이 있는 거주자",
    "회사원 -> 근로소득이 있는 거주자",
    "연봉 -> 종합소득",
    "월급 -> 근로소득",
    "세금 -> 소득세",
    "공제받다 -> 공제를 적용받다",
    "얼마나 내야하나 -> 세액은 얼마인가",
    "계산해줘 -> 계산하면 얼마인가"
]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전을 참고해서 
사용자의 질문을 변경해 주세요. 만약 변경할 필요가 없을경우, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해 주세요.
사전 : {keyword_dict}
질문 : {{question}}""")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0,
                api_key=OPENAI_API_KEY)
keyword_chain = prompt | llm | StrOutputParser()
query = "연봉 5000만원인 회사원의 소득세는 얼마예요?"
normalized_query = keyword_chain.invoke({"question":query})
normalized_query

'종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 세액은 얼마인가?'

In [103]:
keyword_chain.invoke({"question":"월급이 높은 남자가 있습니다"})

'근로소득이 있는 거주자가 높은 소득을 가지고 있습니다.'

In [104]:
query = "종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 얼마예요?"
retriever = vector_database.as_retriever(
    search_type="similarity",
    search_kwargs = {
        "k":10,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)
documents = retriever.invoke(query)

In [105]:
# 5. 참조 조항을 추출하는 함수
def extract_articles_from_docs(documents):
    """
    검색된 문서들에서 article 정보를 추출하여 리스트로 반환
    
    Args:
        documents: retriever에서 반환된 Document 객체 리스트
    
    Returns:
        list: 중복 제거된 조항 리스트 (예: ['제4조', '제16조', '제45조'])
    """
    articles = []
    
    for doc in documents:
        # metadata에서 article 정보 추출 (없으면 기본값)
        article = doc.metadata.get('article', '조항없음')
        
        # article이 문자열이면 그대로 사용, 아니면 처리
        if isinstance(article, str) and article != '조항없음':
            # 「4조, 16조, 45조」 형태를 분리
            article = article.replace('「', '').replace('」', '')
            article_list = [a.strip() for a in article.split(',')]
            articles.extend(article_list)
    
    # 중복 제거 및 정렬
    unique_articles = list(set(articles))
    unique_articles.sort(key=lambda x : int(x[:-1]))
    # '조' 앞에 '제'를 붙여서 표준 형식으로 변환
    # unique_articles = ['조항없음']
    final_articles = ["제"+article for article in unique_articles if article!='조항없음']
    if final_articles:
        return "소득세법 " + ", ".join(final_articles)
    else:
        return unique_articles[0]
extract_articles_from_docs(documents)

'소득세법 제1조, 제2조, 제3조, 제4조, 제5조, 제6조, 제9조, 제10조, 제11조, 제12조, 제13조, 제14조, 제16조, 제17조, 제19조, 제20조, 제21조, 제22조, 제24조, 제26조, 제27조, 제29조, 제31조, 제35조, 제37조, 제39조, 제41조, 제45조, 제46조, 제47조, 제48조, 제49조, 제50조, 제51조, 제52조, 제55조, 제56조, 제57조, 제58조, 제59조, 제95조, 제99조, 제100조, 제119조, 제122조, 제129조, 제133조, 제146조, 제156조, 제160조, 제251조'

## <span style="color:rgb(0,100,200);">title 기반 rerank 함수</span>

In [106]:
documents = retriever.invoke(normalized_query)

In [107]:
print("관련 문서")
for i, doc in enumerate(documents):
    chunk_id = doc.metadata.get('chunk_id')
    category = doc.metadata['category']
    article  = doc.metadata.get('article', '조항없음')
    title = doc.metadata.get('title')
    print(f"{i+1}.{chunk_id}:{article} {category} - {title}")

관련 문서
1.chunk_046:「14조, 17조, 55조, 56조」 ['세율계산', '이자배당', '공제감면', '과세방식', '납세의무', '사업소득', '과세기간'] - 제55조세율 거주자의 종합소득에 대한 소득세 세율 및 퇴직소득 산출세액 계산 방법.
2.chunk_038:「4조, 16조, 45조, 46조, 47조, 133조, 156조」 ['공제감면', '근로소득', '세율계산', '신고납부', '납세의무', '이자배당', '사업소득', '연금소득', '과세기간', '과세방식'] - 소득세법 제46조 및 제46조의2의 핵심 제목: 채권 등에 대한 소득금액의 계산 특례 및 중도 해지로 인한 이자소득금액 계산의 특례
3.chunk_005:「6조, 9조, 10조, 11조, 12조, 99조」 ['납세의무', '사업소득', '세율계산', '근로소득', '비과세', '신고납부', '과세기간'] - 소득세법 제10조납세지의 변경신고 및 제11조과세 관할와 제12조비과세소득 관련 규정 요약.
4.chunk_002:「2조, 3조, 4조, 5조, 9조, 17조, 100조, 119조, 251조」 ['납세의무', '이자배당', '양도소득', '근로소득', '사업소득', '연금소득', '기타소득', '신고납부', '과세기간'] - 공동 소유 자산의 양도소득금액 계산 및 납세의무
5.chunk_039:「22조, 47조, 48조, 49조」 ['공제감면', '근로소득', '납세의무', '연금소득', '과세기간', '과세방식'] - 제5관 근로소득공제, 연금소득공제 및 퇴직소득공제
6.chunk_009:「12조, 13조, 14조, 16조, 17조, 19조, 20조, 21조, 22조, 24조, 26조, 27조, 29조, 31조, 35조, 37조, 39조, 41조, 46조, 47조, 50조, 51조, 52조, 129조」 ['세율계산', '근로소득', '이자배당', '공제감면', '납세의무', '사업소득', '기타소득', '연금소득', '과세기간', '과세방식'] - 소득세법 제14조: 

In [108]:
import cohere
import numpy as np
def rerank_by_title(query: str, documents: list, top_k: int = 4) -> list:
    """
    Cohere의 Rerank API를 사용한 문서 재정렬
    Args:
        query: 사용자 질문
        documents: 검색된 Document 객체 리스트
        top_k: 반환할 문서 개수 (기본값은 4)    
    Returns:
        list: title 유사도 기준으로 재정렬된 Document 리스트
    """
    # Cohere 클라이언트 초기화
    co = cohere.Client(api_key=COHERE_API_KEY)
    # Document 객체에서 텍스트 추출 (뭐 title이 있으니 title도 포함함)
    docs_text = [doc.metadata.get('title', '') +" "+doc.page_content for doc in documents]
    # docs_text = [doc.page_content for doc in documents]
    # Cohere Rerank API 호출
    results = co.rerank(
        model="rerank-multilingual-v3.0",  # 한국어 지원 모델
        query=query,
        documents=docs_text,
        top_n=top_k,
        # return_documents=False  # 문서 내용은 반환하지 않음 (인덱스만)
    )    
    # 재정렬된 문서 반환
    reranked_docs = [documents[r.index] for r in results.results]
    
    return reranked_docs

In [109]:
query = "종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 세액은 얼마인가요?"
docs = rerank_by_title(query, documents, top_k=3)

In [110]:
print("관련 문서")
for i, doc in enumerate(docs):
    chunk_id = doc.metadata.get('chunk_id')
    category = doc.metadata['category']
    article  = doc.metadata.get('article', '조항없음')
    title = doc.metadata.get('title')
    print(f"{i+1}.{chunk_id}:{article} {category} - {title}")

관련 문서
1.chunk_046:「14조, 17조, 55조, 56조」 ['세율계산', '이자배당', '공제감면', '과세방식', '납세의무', '사업소득', '과세기간'] - 제55조세율 거주자의 종합소득에 대한 소득세 세율 및 퇴직소득 산출세액 계산 방법.
2.chunk_039:「22조, 47조, 48조, 49조」 ['공제감면', '근로소득', '납세의무', '연금소득', '과세기간', '과세방식'] - 제5관 근로소득공제, 연금소득공제 및 퇴직소득공제
3.chunk_038:「4조, 16조, 45조, 46조, 47조, 133조, 156조」 ['공제감면', '근로소득', '세율계산', '신고납부', '납세의무', '이자배당', '사업소득', '연금소득', '과세기간', '과세방식'] - 소득세법 제46조 및 제46조의2의 핵심 제목: 채권 등에 대한 소득금액의 계산 특례 및 중도 해지로 인한 이자소득금액 계산의 특례


In [111]:
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])
# 6. RAG 체인 구성 (LCEL 방식)
prompt_chain = prompt | llm  | StrOutputParser()


# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
result = prompt_chain.invoke({"context":format_documents(docs), "query":normalized_query})
print(result + "\n*위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다*")

종합소득 5,000만원인 근로소득이 있는 거주자의 소득세는 84만원 + (1,400만원을 초과하는 금액의 15%)로 계산됩니다. 5,000만원에서 1,400만원을 초과하는 금액은 3,600만원이므로, 3,600만원의 15%는 540만원입니다. 따라서 총 소득세는 84만원 + 540만원 = 624만원입니다.
*위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다*


In [112]:
# 11. 재사용 가능한 함수로 만들기
def ask_with_reference_rerank(query: str, k: int = 15, top_k:int=4):
    """
    질문에 답변하고 참조 조항을 함께 반환하는 함수
    
    Args:
        question: 사용자 질문
        k: 검색할 문서 개수 (기본값 3)
    query, 표준화된 query, 생성된 답변, 참조조항 출력
    """
    # 질문 표준화
    normalized_query = keyword_chain.invoke({"question": query})
    # Retriever 생성
    filter_condition = {"category": {"$in": categorize_content(normalized_query)}}
    retriever = vector_database.as_retriever(
        search_type="similarity",
        search_kwargs={"k": k, "filter": filter_condition}
    )
    # 문서 검색
    documents = retriever.invoke(normalized_query)
    # query와 유사도가 높은 title로 RERANK된 문서 검색
    reranked_documents = rerank_by_title(query=normalized_query, documents=documents, top_k=top_k)
    
    # 참조 조항 추출
    referenced_articles = extract_articles_from_docs(reranked_documents)
    
    # 프롬프트 템플릿
    template = f"""당신은 최고의 한국 소득세 전문가입니다.
    다음 문맥을 참고하여 질문에 답하세요.
    답을 모르면 모른다고 답하세요.
    최대 3문장으로 간결하게 답변하세요.
    질문 : {{query}}
    문맥 : {{context}}
    답변 : """
    prompt = ChatPromptTemplate.from_template(template)
    # 5. 검색된 document를 텍스트로 변환하는 함수
    # 6. RAG 체인 구성 (LCEL 방식)
    prompt_chain = prompt | llm  | StrOutputParser()


    # 7. 실행
    answer = prompt_chain.invoke({"context":format_documents(reranked_documents), "query":normalized_query})
    print("\n" + "="*60)
    print(f"질문: {query}")
    print(f"(표준화: {normalized_query})")
    print("="*60)
    print("\n☑️ 답변:", answer)
    print("\n📌 참조:", referenced_articles)
    print("\n* 위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다 *\n")

In [96]:
# 12. 사용 예시

# 예시 1
ask_with_reference_rerank("연봉 5천만원인 직장인의 소득세는 얼마인가요?")


질문: 연봉 5천만원인 직장인의 소득세는 얼마인가요?
(표준화: 연봉 5천만원인 근로소득이 있는 거주자의 소득세는 얼마인가요?)

☑️ 답변: 연봉 5천만원인 근로소득이 있는 거주자의 소득세는 84만원 + (1,400만원을 초과하는 금액의 15%)로 계산됩니다. 5천만원에서 1,400만원을 초과하는 금액은 3,600만원이므로, 3,600만원의 15%는 540만원입니다. 따라서 총 소득세는 84만원 + 540만원 = 624만원입니다.

📌 참조: 소득세법 제2조, 제6조, 제9조, 제10조, 제11조, 제12조, 제14조, 제16조, 제17조, 제20조, 제21조, 제22조, 제47조, 제48조, 제49조, 제55조, 제56조, 제99조, 제127조, 제129조

* 위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다 *



In [97]:
# 예시 2
ask_with_reference_rerank("사업소득이 있는 경우 필요경비는 어떻게 계산하나요?")

# 예시 3
ask_with_reference_rerank("양도소득세율은 얼마인가요?")


질문: 사업소득이 있는 경우 필요경비는 어떻게 계산하나요?
(표준화: 사업소득이 있는 경우 필요경비는 어떻게 계산하나요?)

☑️ 답변: 사업소득이 있는 경우 필요경비는 해당 과세기간의 총수입금액에 대응하는 통상적인 비용의 합계액으로 계산합니다. 또한, 이전 과세기간에 확정된 비용 중 해당 과세기간에 필요경비로 계상하지 않은 금액만 포함됩니다. 필요경비의 계산에 필요한 사항은 대통령령으로 정해집니다.

📌 참조: 소득세법 제8조, 제27조, 제28조, 제32조, 제33조, 제52조, 제57조, 제62조, 제70조, 제94조, 제106조, 제114조, 제160조, 제161조

* 위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다 *


질문: 양도소득세율은 얼마인가요?
(표준화: 양도소득세율은 얼마인가요?)

☑️ 답변: 양도소득세율은 자산의 종류와 보유 기간에 따라 다릅니다. 예를 들어, 주택의 경우 보유 기간이 1년 미만일 때 70%, 1년 이상 2년 미만일 때 60%의 세율이 적용됩니다. 구체적인 세율은 자산의 종류에 따라 다르므로, 해당 자산에 대한 세율을 확인해야 합니다.

📌 참조: 소득세법 제2조, 제18조, 제20조, 제35조, 제52조, 제55조, 제67조, 제94조, 제95조, 제96조, 제97조, 제103조, 제104조

* 위의 답변은 AI에 의해 작성된 답변이므로 약간의 차이가 날 수 있습니다 *

